: В модели Bag of Words слова просто подсчитываются, поэтому очень частые слова получают огромные значения частоты, «заглушая» более редкие, но важные по смыслу термины. TF-IDF решает эту проблему путем взвешивания. Алгоритм уменьшает вес слов, которые встречаются почти во всех документах, и увеличивает вес уникальных слов, характерных для конкретного текста.

лучшим выбором будет BERT. Модель Word2Vec создает один статический вектор для слова независимо от контекста. BERT же является контекстно-зависимой моделью: она анализирует все предложение целиком и выдаст абсолютно разные векторы для слова «бас» в зависимости от окружающих слов.

Cosiine similarity это метрика которая измеряет угол между векторами в многомерном пространстве. Если угол 0 то косинус = 1, что значит что слова имеют очень идентичное значение, в то время как кос 180 = 0, что означает что слова не имеют никакого отношения друг к другу.

["Don", "t"], ["Don't"], ["Do", "n't"]

In [1]:
import re
from sklearn.feature_extraction.text import CountVectorizer

texts = [
    "Алматы әдемі қала.",
    "Астана - бас қала!",
    "Алматы мен Астана үлкен қалалар."
]

cleaned_texts = [re.sub(r'[^\w\s]', '', text.lower()) for text in texts]

# BoW
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(cleaned_texts)

print("Словарь:", vectorizer.get_feature_names_out())
print("Матрица частот:\n", bow_matrix.toarray())

Словарь: ['алматы' 'астана' 'бас' 'мен' 'қала' 'қалалар' 'үлкен' 'әдемі']
Матрица частот:
 [[1 0 0 0 1 0 0 1]
 [0 1 1 0 1 0 0 0]
 [1 1 0 1 0 1 1 0]]


В этой задаче мы очиститли текст для того чтоб машина не воспринимала слово қала как три разных слова. Поэтому для этого мы произвели очистку через re.sub и .lower(). Мы вывели матрицу частот для того чтоб видеть как машина переводит слова в числа.

In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_texts)

words = np.array(tfidf_vectorizer.get_feature_names_out())
mean_tfidf = tfidf_matrix.mean(axis=0).A1

top_3_indices = mean_tfidf.argsort()[-3:][::-1]
top_3_words = words[top_3_indices]

print("Топ-3 самых важных слова по TF-IDF:", top_3_words)

Топ-3 самых важных слова по TF-IDF: ['қала' 'алматы' 'астана']


Здесь мы использовали TfidfVectorizer. Мы сравниваем как часто встрчеаются слова (чем чаще тем важнее), после смотрим в скольких документах это слово вчтречается (чем чаще тем ценность падает). В этой задаче модель посчитала вес каждого слова, mean(axis=0) усредняет все слова и отсортировав мы находим топ 3 по важности слова.

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

sentences = ["Маған бағдарламалау ұнайды.", "Код жазу менің хоббиім."]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

embeddings = outputs.last_hidden_state.mean(dim=1).numpy()

similarity = cosine_similarity([embeddings[0]], [embeddings[1]])
print(f"Семантическое сходство: {similarity[0][0]:.4f}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-multilingual-cased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Семантическое сходство: 0.5507


В этой задачи мы использовали модель берт потому что она понимает казахский язык.

AutoTokenizer.from_pretrained(model_name) -- модель разбивает текст на токены и переводит на свой айди.

каждое слово получает свой вектор которые описываюют контекст и смысл слова в каждом предложений.


.mean(dim=1) -- для того чтоб получить вектор для всего предложения. модель берт может только заполучать вектора для слов. поэтому машина высчитывает среднию арифметическую векторов для получения вектора для предложений.

и использууем метрику косинус симилирати чтобы выяивить семантическое сходство.  у нас есть два веектора, представляющие смысл двух предложений, и мы измеряем угол между ними. чем ближе значение к одному тем схоже смысл предложений.